<a href="https://colab.research.google.com/github/artemkush1/ML_predict_CS-GO_matches/blob/master/data_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pprint as pp
import requests
import datetime
from bs4 import BeautifulSoup
from re import sub
import re
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

pr = pp.PrettyPrinter()
prefix = 'https://www.hltv.org'
max_rank = 1000
last_months = 6

In [2]:
import time
proxyDict = { 
              "http": 'http://193.169.5.14:39602',
            }

def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    got_req = requests.get(url, headers=headers, proxies=None).text
    time.sleep(0.15)
    return BeautifulSoup(got_req, "lxml")

# Functions for each map

In [187]:


class MatchHolder:
    def __init__(self, match_url):
        self.match_url = match_url
        self.match_page = get_parsed_page(match_url)
        self.team1_url, self.team2_url = self._get_teams_url()
        self.team1_stat_url, self.team2_stat_url = self._get_full_team_stat_page(self.team1_url),\
                                         self._get_full_team_stat_page(self.team2_url)
        self.match_date = datetime.datetime.utcfromtimestamp(int(self._get_unixtime_match()))
        self.maps_played_with_score = self.get_score_on_each_map()
        self.history_h2h = self.get_history_h2h()
        self.team1_page, self.team2_page = get_parsed_page(self.team1_url), get_parsed_page(self.team2_url)
        self.team1_rank, self.team2_rank = self.get_team_rank(self.team1_page), self.get_team_rank(self.team2_page)
        self.event_page = self._get_event_page()
        self.prize_pool = self.get_prize_pool()
        self.count_teams_on_tour = self.get_teams_on_tour()
        self.players_url = self.get_players_from_match_url()
        (self.players_stats_last3m, self.players_stats_last6m) = self.get_stats_each_player()
         #self.players_stats_last1m,
         #self.players_stats_last9m,
         #self.players_stats_last12m...
        self.maps_team1, self.maps_team2 = self.get_maps_url(self.team1_stat_url), self.get_maps_url(self.team2_stat_url)
        self.stat_maps_team1, self.stat_maps_team2 = self.get_maps_stat_dict(self.maps_team1, 'team1',
                                                            skip_params=['Wins / draws / losses'],
                                                            maps_need=[i for i in self.maps_played_with_score][0]), \
                                                     self.get_maps_stat_dict(self.maps_team2, 'team2',
                                                            skip_params=['Wins / draws / losses'],
                                                            maps_need=[i for i in self.maps_played_with_score[0]])
        self.last5matches = self.get_last5_matches_v1_2teams()
        self.pistol_rounds = self.get_pistols_rounds()
        
    def _get_teams_url(self):
        return [prefix + self.match_page.find_all('div', {"class": "team"})[0].find('a')['href'],
                prefix + self.match_page.find_all('div', {"class": "team"})[1].find('a')['href']]
    
    def _get_full_team_stat_page(self, team_url):
        return f'https://www.hltv.org/stats/teams/maps{team_url.split("https://www.hltv.org/team")[-1]}'
    
    def _get_unixtime_match(self):
        return self.match_page.find('div', {"class": "timeAndEvent"}).find('div', {"class": "time"})['data-unix'][:-3]
    
    def _get_event_page(self):
        return get_parsed_page(prefix + self.match_page.find('div', {'class': 'event'}).find('a')['href'])
    
    # return {maps: score on map}, total-maps-counter
    def get_score_on_each_map(self):
        def _get_maps_():
            maps_score = []
            maps_obj = self.match_page.find('div', {"class": "flexbox-column"}).find_all('div', {"class": "mapholder"})
            for i in maps_obj:
                tmp = i.find('div', {"class": "played"})
                if tmp:
                    maps_score += [i]
            return maps_score

        maps_obj = _get_maps_()
        # print(len(maps_obj))
        maps = {}
        i = 0
        for map_ in maps_obj:
            tmp1 = int(map_.find('div', {"class": "results-left"}).find('div', {"class": "results-team-score"}).text)
            tmp2 = int(map_.find('span', {"class": "results-right"}).find('div', {"class": "results-team-score"}).text)
            if len([tmp1, tmp2]) != 2:
                print("Problems in get_score()")
            maps[map_.find('div', {"class": "mapname"}).text] = [tmp1, tmp2]
            i += 1
        return maps, i
    
    # get history_h2h: expect 3 params
    def get_history_h2h(self):
        tmp = [int(i.find('div', {'class': 'bold'}).text) for i in
               self.match_page.find_all('div', {'class': 'flexbox-center'})]
        if len(tmp) != 3:
            print("Problems in get_history_h2h()")
        return tmp
    
    # get teams_rank: expect 1 param
    def get_team_rank(self, team):
        try:
            tmp = max_rank - int(team.find('div', {'class': 'profile-team-stats-container'}).
                                 find_all('span', {'class': "right"})[0].text[1:])
            if not tmp:
                return 0
            return tmp
        except:
            return 0
    
    # get prize_pool: expect 1 param
    def get_prize_pool(self):
        try:
            tmp = int(sub(r'[^\d.]', '', self.event_page.find('td', {'class': 'prizepool'}).text))
            if not tmp:
                return 0
            return tmp
        except:
            return 0
        
    # get teams_on_tour: expect 1 param
    def get_teams_on_tour(self):
        tmp = int(re.findall(r"[-+]?\d*\.\d+|\d+", self.event_page.find('td', {'class': 'teamsNumber'}).text)[0])
        if tmp:
            return tmp
        return 0
    
    # expect 2 * 5 params
    def get_players_from_match_url(self):
        """tmp = [[prefix + '/stats/players' + (i.find('a')['href'].split('/player')[-1])
                 for i in k.find_all('td', {'class': 'player'})][:5]
                for k in self.match_page.find_all('div', {'class': 'lineup standard-box'})]"""
        tmp = {}
        all_pl = self.match_page.find_all('div', {'class': 'lineup standard-box'})
        for k in range(2):
            for i in range(5):
                try:
                    tmp[f'team{k}_pl{i}'] = prefix + '/stats/players' + \
                            (all_pl[k].find_all('td', {'class': 'player'})[i].find('a')['href'].split('/player')[-1])
                except:
                    tmp[f'team{k}_pl{i}'] = ""
        return tmp

    # expect 19 params in dict
    def _get_player_many_stats(self, player, index):
        table_up = player.find('div', {'class': 'statistics'}).find_all('div', {'class': 'stats-row'})
        res = {}
        for i in table_up:
            tmp1 = i.find_all('span')[-1].text
            if tmp1[-1] == '%':
                tmp1 = float(tmp1[:-1]) / 100
            elif tmp1[-1] == '-':
                tmp1 = '0'
            res[i.find_all('span')[0].text + index] = tmp1
        table_down = player.find('div', {'class': 'featured-ratings-container'}).find_all('div', {'class': 'col-custom'})
        for i in table_down:
            tmp1 = i.find('div', {'class': 'rating-value'}).text
            if tmp1[-1] == '%':
                tmp1 = float(tmp1[:-1]) / 100
            elif tmp1[-1] == '-':
                tmp1 = '0'
            res[i.find('div', {'class': 'rating-description'}).text + index] = tmp1
        if len(res) != 19:
            print("Problems in get_player_many_stats()")
        return res

    # return dict with 19 params
    def _get_player_stat_in_period(self, player_url, date_start, date_end, index):
        player_page = get_parsed_page(player_url + \
                      f'?startDate={date_start.strftime("%Y-%m-%d")}&endDate={date_end.strftime("%Y-%m-%d")}')
        return self._get_player_many_stats(player_page, index)
    
    # return 5 big dict: in each 5 players with 19 params
    def get_stats_each_player(self):
        _1m = {}
        _3m = {}
        _6m = {}
        #_9m = {}
        #_12m = {}
        for dict_, months in zip([_1m, _3m, _6m],
                     [1, 3, 6]):
            for i in self.players_url:
                # print(i)
                try:
                    dict_[i] = [self._get_player_stat_in_period(self.players_url[i],
                                                                self.match_date - relativedelta(months=months),
                                                                self.match_date - relativedelta(days=1),
                                                                '_'+i)]
                except Ex:
                    dict_[i] = [0 for t in range(19)]
        return _1m, _3m  # , _6m, _9m, _12m

    # get url on maps
    def get_maps_url(self, team_url_maps):
        dict_ = {}
        # print(team_url_maps)
        maps = get_parsed_page(team_url_maps).find('div', {'class': 'two-grid'}).find_all('div', {'class': 'map-pool'})
        for i in maps:
            dict_[i.find('img', {'class': 'map-pool-map'})['title']] = prefix + i.find('a')['href']
        return dict_
    
    # expect (10 - skip_params) parameters in dict
    def _get_map_many_stats(self, map_page, index, skip_params):
        table = map_page.find('div', {'class': 'stats-rows'}).find_all('div', {'class': 'stats-row'})
        stats = {}
        for k in table:
            if k.find_all('span')[0].text not in skip_params:
                tmp = k.find_all('span')[-1].text
                if tmp[-1] == '%':
                    tmp = float(tmp[:-1]) / 100
                elif tmp[-1] == '-':
                    tmp = 0
                stats[k.find_all('span')[0].text + index] = float(tmp)
        if len(stats) != (10 - len(skip_params)):
            print("Problems in get_stats_on_each_map() (10 params)")
        return stats
    
    # expect dict
    def _get_maps_stat_in_period(self, match_url, date_start, date_end, index, skip_params):
        map_page = get_parsed_page(match_url + \
                      f'?startDate={date_start.strftime("%Y-%m-%d")}&endDate={date_end.strftime("%Y-%m-%d")}')
        return self._get_map_many_stats(map_page, index, skip_params)

    # expect 5 big DICTS with (10 - skip_params) parameters in each map
    def get_maps_stat_dict(self, maps, index, skip_params=[], maps_need=None):
        # self.maps_team1, self.maps_team2
        _1m = {}
        _3m = {}
        _6m = {}
        #_9m = {}
        #_12m = {}
        for dict_, months in zip([_1m, _3m, _6m],
                     [1, 3, 6, 9, 12]):
            for i in maps:
                # print(i)
                if i in maps_need:
                    dict_[i] = [self._get_maps_stat_in_period(maps[i],
                               self.match_date - relativedelta(months=months),
                               self.match_date - relativedelta(days=1),
                               '_'+index, skip_params)]
                elif maps_need is None:
                    dict_[i] = [self._get_maps_stat_in_period(maps[i],
                               self.match_date - relativedelta(months=months),
                               self.match_date - relativedelta(days=1),
                               '_'+index, skip_params)]
        return [_1m, _3m]  # , _6m, _9m, _12m


    # get lats 5 matches for both teams: expect 5 params in 2 dicts in 2 other dicts
    def get_last5_matches_v1_2teams(self):
        def get_url_opponent(url1, i):
            return prefix + url1.find_all('tr', {'class': 'table'})[i].find('a', {'class': 'text-ellipsis'})['href']

        def get_type_match(url1, i):
            return url1.find_all('tr', {'class': 'table'})[i].find('a', {'data-link-tracking-page': 'Matchpage'}).text

        def get_score_array(url1, i):
            return [int(k) for k in
                    url1.find_all('tr', {'class': 'table'})[i].find('td', {'class': 'spoiler'}).text.split(' - ')]

        def get_all(table):
            team_ranks = {}
            team_scores = {}
            for i in range(5):
                try:
                    rank_ = self.get_team_rank(get_parsed_page(get_url_opponent(table, i)))
                    score1_, score2_ = get_score_array(table, i)
                    team_ranks[f'opponent_rank_from_5_last_matches_{i}'] = rank_
                    # print('scores last 5 matches:', score1_, score2_)
                    if score1_ > score2_:
                        team_scores[f'win/loss_from_5_last_matches_{i}'] = 1
                    else:
                        team_scores[f'win/loss_from_5_last_matches_{i}'] = -1
                except Exception as e:
                    print('Problems in get_last5_matches_v1_2teams()', e)
                    team_ranks[f'opponent_rank_from_5_last_matches_{i}'] = '0'
                    team_scores[f'win/loss_from_5_last_matches_{i}'] = 0
            return team_ranks, team_scores
        last5m = {}
        tables = (self.match_page.find('div', {'class': 'past-matches'}).
                 find_all('div', {'class': 'half-width'}))
        last5m['team1'] = [get_all(tables[0])]
        last5m['team2'] = [get_all(tables[1])]
        # last5_matches_2teams = [*types, *teams_rank, *scores]
        if len(last5m) != 2:
            print('Problems in get_last5_matches_v1_2teams()')
        return last5m

    def get_pistols_rounds(self):
        def _get_maps_():
            maps_score = []
            maps_obj = self.match_page.find('div', {"class": "flexbox-column"}).find_all('div', {"class": "mapholder"})
            for i in maps_obj:
                tmp = i.find('div', {"class": "played"})
                if tmp:
                    maps_score += [i]
            return maps_score

        maps_obj = _get_maps_()
        # print(len(maps_obj))
        maps = {}
        i = 0
        for map_ in maps_obj:
            tmp = prefix + map_.find('div', {"class": "results-center"}).\
                                find('a', {"class": "results-stats"})['href']
            page_stat = get_parsed_page(tmp)
            tables = page_stat.find_all('div', {"class": "round-history-half"})[:2]
            pistols = {}
            n = 1
            for k in tables:
                if k.find('img', {"class": "round-history-outcome"})['src'] !=\
                            '//static.hltv.org/images/scoreboard/emptyHistory.svg':
                    pistols[f'pistol_round_{n}'] = 1
                else:
                    pistols[f'pistol_round_{n}'] = -1
                n += 14
            maps[map_.find('div', {"class": "mapname"}).text] = [pistols]
        return maps



# get_metches_url(filename='matches.txt', pages_with_results=[0, 1])
# cl = MatchHolder('https://www.hltv.org/matches/2341715/vitality-vs-dignitas-blast-premier-spring-2020-europe-showdown')
# print(cl.stat_maps_team1)
# cl.pistol_rounds


In [215]:
# download all urls matches
def get_matches_url(filename=None, pages_with_results=[0]):
    all_matches = []
    for i in pages_with_results:
        if i == 0:
            results = get_parsed_page('https://www.hltv.org/results')
        else:
            results = get_parsed_page(f'https://www.hltv.org/results?offset={i}00')
        # list of matches
        all_matches += [prefix + url_.find('a', {"class": "a-reset"})['href'] for url_ in
                        results.find('div',
                                     {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}).
                        find_all('div', {"class": "result-con"})]
    all_matches = np.array(all_matches)
    # print(all_matches)
    if filename is None:
        return all_matches
    else:
        np.save(filename, np.array(all_matches))


get_matches_url(filename='all_matches', pages_with_results=[i for i in range(100)])

In [188]:
# cl = MatchHolder('https://www.hltv.org/matches/2341715/vitality-vs-dignitas-blast-premier-spring-2020-europe-showdown')

In [216]:
def get_pist_r(stat, name):
    res = []
    lb = []
    for i in stat[name][0]:
        res += [stat[name][0][i]]
        lb += [i]
    return res, lb
    


def get_big_stat_map(stat, index, name):
    res = []
    lb = []
    for i in stat[index][name][0]:
        res += [stat[index][name][0][i]]
        lb += [i]
    return res, lb

def get_big_player_stat(stat):
    labels = []
    res = []
    for player in stat:
        for k in stat[player][0]:
            labels += [k]
            res += [stat[player][0][k]]
    return res, labels

def get_last_5m(stat):
    labels = []
    res = []
    for i in stat:
        for k in stat[i][0]:
            for j in k:
                labels += [j]
                res += [k[j]]
    return res, labels

def prepare_maps_played(stat):
    dict_ = {}
    for i in stat:
        dict_[i] = ['score1', 'score2']
    return stat, dict_


In [227]:


FINAL_DICT = {0: None,
              1: None}


def final_def(cl):
    # Взял статистику по всем картам и количество сыгранных карт
    maps_played, maps_total = cl.maps_played_with_score  # added
    maps_played_score, lab_maps_played = prepare_maps_played(maps_played)  # added


    ranks_score_last5, labels_last5 = get_last_5m(cl.last5matches)  # added


    data_1m = []
    data_2m = []
    data_3m = []


    # Рассматриваем одну из карт
    for name in maps_played_score:
        score = maps_played_score[name]  # added
        lbscore = lab_maps_played[name]  # added
        # Сохраняем данные за разный период
        for i, big_data_map in zip([0, 1], [cl.players_stats_last3m,
                                            cl.players_stats_last6m]):
            player_stat, lab_pl_stat = get_big_player_stat(big_data_map)  # OK  # added
            maps_stat1, lab_map_stat1 = get_big_stat_map(cl.stat_maps_team1, i, name)  # OK  # added
            maps_stat2, lab_map_stat2 = get_big_stat_map(cl.stat_maps_team2, i, name)  # OK  # added
            pist, lbpist = get_pist_r(cl.pistol_rounds, name)

            final_data = np.array\
            ([cl.match_url, *score, maps_total, cl.maps_played_with_score[1], *pist,
              *cl.history_h2h,
             cl.team1_rank, cl.team2_rank,
             cl.prize_pool, cl.count_teams_on_tour, 
             *ranks_score_last5, *player_stat, *maps_stat1, *maps_stat2
            ])

            labels = np.array\
            (['url', *lbscore, 'maps_total', 'maps_total', *lbpist,
              'history_t1', 'history_draw', 'history_t2',
             'team1_rank', 'team2_rank',
             'prize_pool', 'teams_on_tour',
             *labels_last5, *lab_pl_stat, *lab_map_stat1, *lab_map_stat2
            ])

            if FINAL_DICT[i] is None:
                FINAL_DICT[i] = pd.DataFrame([final_data],
                                             columns=labels)
            else:
                FINAL_DICT[i] = pd.concat([FINAL_DICT[i],
                                           pd.DataFrame([final_data],
                                                        columns=labels)])


In [ ]:
matches = np.load('all_matches.npy', allow_pickle=True)

for match in matches:
    print(match)
    cl = MatchHolder(match)
    try:
        final_def(cl)
    except Exception as e:
        print(e)
    for i in FINAL_DICT:
        FINAL_DICT[i].to_csv(f'df_{i}.csv')



https://www.hltv.org/matches/2341894/havu-vs-endpoint-home-sweet-home-cup-7
https://www.hltv.org/matches/2341832/fate-vs-espada-winners-league-season-4-europe


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_save = np.array(data)
df = pd.DataFrame(data=data_save[1:,:], columns=data_save[0, :])
df.to_csv('/content/drive/My Drive/Colab Notebooks/cs-go_matches/df.csv')
